In [ ]:
import pandas as pd
import json
from difflib import SequenceMatcher
from collections import defaultdict

In [ ]:
# Paramètres
POLISH_DIGRAPHS = ["ch", "cz", "dz", "dź", "dż", "rz", "sz", "eu", "ou"]
ENDING_SUFFIXES = ["ą", "a", "u", "em", "ę", "i", "y", "ej", "ich", "ego", "e"]

In [1]:
# Fonctions
def split_phonetic_units(word):
    i, units = 0, []
    while i < len(word):
        matched = False
        for digraph in sorted(POLISH_DIGRAPHS, key=len, reverse=True):
            if word[i:i+len(digraph)] == digraph:
                units.append(digraph)
                i += len(digraph)
                matched = True
                break
        if not matched:
            units.append(word[i])
            i += 1
    return units

def strip_ending(correct, incorrect):
    if correct.endswith("ą") and incorrect.endswith(("on", "om", "o")):
        return correct
    if correct.endswith("ę") and incorrect.endswith("en"):
        return correct
    if correct.endswith("o") and incorrect.endswith("eu"):
        return correct
    if correct.endswith("eu") and incorrect.endswith("o"):
        return correct
    for suffix in sorted(ENDING_SUFFIXES, key=len, reverse=True):
        if correct.endswith(suffix):
            return correct[:-len(suffix)]
    return correct

def find_differences(correct, incorrect):
    correct_base = strip_ending(correct, incorrect)
    incorrect_base = strip_ending(incorrect, correct)
    correct_units = split_phonetic_units(correct_base)
    incorrect_units = split_phonetic_units(incorrect_base)
    matcher = SequenceMatcher(None, correct_units, incorrect_units)
    differences = []
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag != 'equal':
            differences.append({
                "correct": "".join(correct_units[i1:i2]) if i1 != i2 else "-",
                "incorrect": "".join(incorrect_units[j1:j2]) if j1 != j2 else "-"
            })
    return differences

# Lecture CSV de base
df = pd.read_csv("statistiques_erreurs_par_apprenant.csv")

# Création d'une version "aplatie"
rows = []
for _, row in df.iterrows():
    correct_word = str(row["Forme correcte"])
    incorrect_word = str(row["Forme erronée (manuel)"])
    differences = find_differences(correct_word, incorrect_word)

    if not differences:
        continue

    for diff in differences:
        new_row = {
            "Forme correcte": correct_word,
            "Forme erronée (manuel)": incorrect_word,
            "Correcte": diff["correct"],
            "Erroné": diff["incorrect"]
        }
        for col in ["FR", "IT", "NL", "UK", "GE", "Nombre d'apprenants"]:
            new_row[col] = row.get(col, 0)
        
        # Dodanie kolumny Apprenants z listą uczniów
        new_row["Apprenants"] = row.get("Apprenants", "")

        rows.append(new_row)

flat_df = pd.DataFrame(rows)

# Ajout d'une colonne clé de groupe basée sur Correcte et Erroné
flat_df["pair_key"] = flat_df["Correcte"] + "|" + flat_df["Erroné"]

# Calcule la priorité de tri pour chaque groupe de paire
pair_priority = flat_df.groupby("pair_key")["Nombre d'apprenants"].transform("max")

# Trie d'abord par priorité du groupe (desc), puis par Nombre d'apprenants (desc)
flat_df_sorted = flat_df.assign(group_priority=pair_priority).sort_values(
    by=["group_priority", "Correcte", "Erroné", "Nombre d'apprenants"],
    ascending=[False, True, True, False]
).drop(columns=["pair_key", "group_priority"])

# Sauvegarde
flat_df_sorted.to_csv("erreurs_prononciation.csv", index=False, encoding="utf-8")

print("Fichier mis à jour : erreurs_prononciation.csv.")


Fichier mis à jour : erreurs_prononciation.csv.


In [2]:
# Création du dictionnaire (correct, manuel) → liste des lettres
diff_dict = defaultdict(list)
for _, row in flat_df.iterrows():
    key = (row["Forme correcte"], row["Forme erronée (manuel)"])
    diff_dict[key].append((row["Correcte"], row["Erroné"]))

# Mise à jour du JSON
with open("alignement_mots_manuel_correct.json", encoding="utf-8") as f:
    data = json.load(f)

for record in data:
    for segment in record["segments"]:
        for mot in segment["segments"]:
            key = (mot.get("correct", ""), mot.get("manuel", ""))
            lettres = diff_dict.get(key, [])
            if lettres:
                for idx, (c, m) in enumerate(lettres, start=1):
                    mot[f"lettre_correct_{idx}"] = c
                    mot[f"lettre_manuel_{idx}"] = m
            else:
                mot["lettre_correct_1"] = ""
                mot["lettre_manuel_1"] = ""

with open("alignement_mots_manuel_correct_prononciation.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
print("Fichier JSON mis à jour : alignement_mots_manuel_correct_prononciation.json")

Fichier JSON mis à jour : alignement_mots_manuel_correct_prononciation.json


In [3]:
# Dictionnaire des ensembles d'apprenants par (Correcte, Erroné)
apprenants_dict = defaultdict(set)

for _, row in flat_df.iterrows():
    key = (row["Correcte"], row["Erroné"])
    apprenants_str = row.get("Apprenants", "")
    if pd.notna(apprenants_str) and apprenants_str.strip() != "":
        apprenants_set = set(x.strip() for x in apprenants_str.split(","))
        apprenants_dict[key].update(apprenants_set)

# Map du premier chiffre vers le pays
country_map = {
    "1": "FR",
    "5": "IT",
    "2": "NL",
    "3": "UK",
    "4": "GE"
}

# Pour chaque paire (Correcte, Erroné), créer un dictionnaire pour compter les apprenants par pays
country_counts = defaultdict(lambda: {"FR": 0, "IT": 0, "NL": 0, "UK": 0, "GE": 0})

for key, apprenants in apprenants_dict.items():
    # Comptage unique par pays
    counted = {"FR": set(), "IT": set(), "NL": set(), "UK": set(), "GE": set()}
    for apprenant in apprenants:
        if apprenant and apprenant[0] in country_map:
            pays = country_map[apprenant[0]]
            counted[pays].add(apprenant)
    for pays in counted:
        country_counts[key][pays] = len(counted[pays])

# Création du résumé avec les totaux mis à jour
summary_rows = []
for key in apprenants_dict.keys():
    total_apprenants = len(apprenants_dict[key])
    row = {
        "Correcte": key[0],
        "Erroné": key[1],
        "FR": country_counts[key]["FR"],
        "IT": country_counts[key]["IT"],
        "NL": country_counts[key]["NL"],
        "UK": country_counts[key]["UK"],
        "GE": country_counts[key]["GE"],
        "Nombre total d'apprenants": total_apprenants
    }
    summary_rows.append(row)

summary = pd.DataFrame(summary_rows)

# Ajout de la colonne "Examples de mots"
mot_dict = defaultdict(set)
for _, row in flat_df.iterrows():
    key = (row["Correcte"], row["Erroné"])
    mot_dict[key].add(row["Forme correcte"])

summary["Examples de mots"] = summary.apply(
    lambda row: ", ".join(sorted(mot_dict[(row["Correcte"], row["Erroné"])])),
    axis=1
)

# Tri par nombre d'apprenants décroissant
summary = summary.sort_values(by="Nombre total d'apprenants", ascending=False)

summary.to_csv("erreurs_prononciation_groupées_par_phonèmes.csv", index=False, encoding="utf-8")
print("Fichier erreurs_prononciation_groupées_par_phonèmes.csv.")

Fichier erreurs_prononciation_groupées_par_phonèmes.csv.


In [4]:
import pandas as pd
import epitran

# Initialisation de l'outil de translittération Epitran pour le polonais
epi = epitran.Epitran('pol-Latn')

# Lecture du fichier original avec les erreurs de prononciation
df = pd.read_csv("erreurs_prononciation_groupées_par_phonèmes.csv")

# Fonction de translittération simple pour une cellule texte
def convert_to_ipa(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    return epi.transliterate(text)

# Fonction de translittération pour la colonne contenant plusieurs mots séparés par des virgules
def convert_examples_to_ipa(examples):
    if pd.isna(examples) or not isinstance(examples, str):
        return ""
    words = [word.strip() for word in examples.split(",")]
    ipa_words = [epi.transliterate(word) for word in words]
    return ", ".join(ipa_words)

# Remplacement des colonnes par leurs équivalents phonétiques
df["Correcte"] = df["Correcte"].apply(convert_to_ipa)
df["Erroné"] = df["Erroné"].apply(convert_to_ipa)
df["Examples de mots"] = df["Examples de mots"].apply(convert_examples_to_ipa)

# Sauvegarde dans un nouveau fichier CSV
df.to_csv("erreurs_prononciation_groupées_par_phonèmes_API.csv", index=False, encoding="utf-8")

print("Fichier sauvegardé : erreurs_prononciation_groupées_par_phonèmes_API.csv.")


Fichier sauvegardé : erreurs_prononciation_groupées_par_phonèmes_API.csv.
